# 使用 Milvus 和 DeepSeek 构建 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

### 依赖与环境

In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


---

In [2]:
import os

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")


### 准备数据

我们使用 Milvus 文档 2.4.x 中的 FAQ 页面作为我们 RAG 中的私有知识库，这是一个简单 RAG 管道的良好数据源。

下载 zip 文件并将文档解压到 `milvus_docs` 文件夹。

**建议在命令行执行下面命令**

我们从 `mfd_docs/` 文件夹加载所有 markdown 文件。对于每个文档，我们简单地使用 "# " 来分割文件中的内容，这样可以大致分离出 markdown 文件中每个主要部分的内容。

In [3]:
from glob import glob

text_lines = []

for file_path in glob("mfd_docs/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [4]:
len(text_lines)

30

### 准备 LLM 和 Embedding 模型

DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。

In [5]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

定义一个 embedding 模型，使用 `milvus_model` 来生成文本嵌入。我们以 `DefaultEmbeddingFunction` 模型为例，这是一个预训练的轻量级嵌入模型。

In [6]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

生成一个测试嵌入并打印其维度和前几个元素。

In [7]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.04836059  0.07163021 -0.01130063 -0.03789341 -0.03320651 -0.01318453
 -0.03041721 -0.02269495 -0.02317858 -0.00426026]


In [8]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

[-0.02752976  0.0608853   0.00388525 -0.00215193 -0.02774976 -0.0118618
 -0.04020916 -0.06023417 -0.03813156  0.0100272 ]


## 将数据加载到 Milvus

### 创建 Collection

In [9]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./mfd_milvus_demo.db")

collection_name = "my_rag_collection"

关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。

检查 collection 是否已存在，如果存在则删除它。

In [10]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

创建一个具有指定参数的新 collection。

如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

`metric_type` (距离度量类型):
     作用：定义如何计算向量之间的相似程度。
     例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
     选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。

 `consistency_level` (一致性级别):
     作用：定义数据写入后，读取操作能多快看到这些新数据。
     例如：
         `Strong` (强一致性): 总是读到最新数据，可能稍慢。
         `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
         `Session` (会话一致性): 自己写入的自己能立刻读到。
         `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
     选择依据：在数据实时性要求和系统性能之间做权衡。

简单来说：
 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [11]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [12]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 30/30 [00:00<00:00, 332002.96it/s]


{'insert_count': 30, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'cost': 0}

## 构建 RAG

### 检索查询数据

我们指定一个关于 Milvus 的常见问题。

In [13]:
question = "物权有哪些"

在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [14]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

让我们看一下查询的搜索结果

In [15]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u7b2c\u4e09\u7ae0 \u5408\u540c\u7684\u53d8\u66f4\u548c\u8f6c\u8ba9\n\n**\u7b2c\u4e94\u767e\u56db\u5341\u516b\u6761** \u5f53\u4e8b\u4eba\u534f\u5546\u4e00\u81f4\uff0c\u53ef\u4ee5\u53d8\u66f4\u5408\u540c\u3002\n\n**\u7b2c\u4e94\u767e\u56db\u5341\u4e5d\u6761** \u5f53\u4e8b\u4eba\u5bf9\u5408\u540c\u53d8\u66f4\u7684\u5185\u5bb9\u7ea6\u5b9a\u4e0d\u660e\u786e\u7684\uff0c\u63a8\u5b9a\u4e3a\u672a\u53d8\u66f4\u3002\n\n**\u7b2c\u4e94\u767e\u4e94\u5341\u6761** \u503a\u6743\u4eba\u53ef\u4ee5\u5c06\u5408\u540c\u7684\u6743\u5229\u5168\u90e8\u6216\u8005\u90e8\u5206\u8f6c\u8ba9\u7ed9\u7b2c\u4e09\u4eba\uff0c\u4f46\u662f\u6709\u4e0b\u5217\u60c5\u5f62\u4e4b\u4e00\u7684\u9664\u5916\uff1a\n\uff08\u4e00\uff09\u6839\u636e\u5408\u540c\u6027\u8d28\u4e0d\u5f97\u8f6c\u8ba9\uff1b\n\uff08\u4e8c\uff09\u6309\u7167\u5f53\u4e8b\u4eba\u7ea6\u5b9a\u4e0d\u5f97\u8f6c\u8ba9\uff1b\n\uff08\u4e09\uff09\u4f9d\u7167\u6cd5\u5f8b\u89c4\u5b9a\u4e0d\u5f97\u8f6c\u8ba9\u3002\n\u503a\u6743\u4eba\u8f6c\u8ba9\u6743\u522

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [16]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [17]:
context

'第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###\n第五章 占有\n\n**第四百七十一条** 占有是指对物事实上的控制和支配。\n\n**第四百七十二条** 占有可以分为直接占有和间接占有。\n直接占有是指直接对物进行控制和支配。\n间接占有是指通过他人对物进行控制和支配。\n\n**第四百七十三条** 占有的取得和消灭，适用本法有关物权设立和消灭的规定。\n\n**第四百七十四条** 占有人合法占有动产的，善意取得人取得该动产所有权。\n占有人非法占有动产的，善意取得人取得该动产所有权。\n\n**第四百七十五条** 占有物毁损、灭失的，占有人应当承担赔偿责任。\n占有人善意占有物的，不承担赔偿责任。\n占有人恶意占有物的，应当承担赔偿责任。\n\n**第四百七十六条** 占有被侵夺的，占有人有权请求返还原物。\n占有物毁损、灭失的，占有人有权请求赔偿损失。\n\n**第四百七十七条** 占有被侵夺的，占有人有权请求返还原物。\n占有物毁损、灭失的，占有人有权请求赔偿损失。\n\n**第四百七十八条** 占有物

In [18]:
question

'物权有哪些'

为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [19]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
<context>
{context}
</context>
<question>
{question}
</question>
"""

In [20]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。\n<context>\n第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###\n第五章 占有\n\n**第四百七十一条** 占有是指对物事实上的控制和支配。\n\n**第四百七十二条** 占有可以分为直接占有和间接占有。\n直接占有是指直接对物进行控制和支配。\n间接占有是指通过他人对物进行控制和支配。\n\n**第四百七十三条** 占有的取得和消灭，适用本法有关物权设立和消灭的规定。\n\n**第四百七十四条** 占有人合法占有动产的，善意取得人取得该动产所有权。\n占有人非法占有动产的，善意取得人取得该动产所有权。\n\n**第四百七十五条** 占有物毁损、灭失的，占有人应当承担赔偿责任。\n占有人善意占有物的，不承担赔偿责任。\n占有人恶意占有物的，应当承担赔偿责任。\n\n**第四百七十六条** 占有被侵夺的，占有人有权请求返还原物。\n占有物毁损、灭失的，占有人有权请求赔偿损失。\n\n**第四百七十

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [21]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，物权主要包括以下几种类型：

1. **占有**（第四百七十一条至第四百八十条）：
   - 对物事实上的控制和支配，分为直接占有和间接占有。
   - 涉及占有的取得、消灭、善意/恶意占有的责任及占有被侵夺时的救济。

2. **地役权**（第三百八十八条至第三百九十八条）：
   - 地役权人有权利用他人的不动产（供役地）以提高自己的不动产（需役地）的效益。
   - 包括地役权的设立、登记、行使、转让、消灭及相关的权利义务。

此外，虽然上下文未明确列出其他物权类型，但根据中国《民法典》物权编的完整内容，物权通常还包括所有权、用益物权（如土地承包经营权、建设用地使用权、宅基地使用权等）和担保物权（如抵押权、质权、留置权）。当前提供的片段中仅详细涉及**占有**和**地役权**（用益物权的一种）。


In [22]:
question = "农村集体经济的财产怎么确权" 

In [23]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [24]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [25]:
context

'第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###\n第五章 占有\n\n**第四百七十一条** 占有是指对物事实上的控制和支配。\n\n**第四百七十二条** 占有可以分为直接占有和间接占有。\n直接占有是指直接对物进行控制和支配。\n间接占有是指通过他人对物进行控制和支配。\n\n**第四百七十三条** 占有的取得和消灭，适用本法有关物权设立和消灭的规定。\n\n**第四百七十四条** 占有人合法占有动产的，善意取得人取得该动产所有权。\n占有人非法占有动产的，善意取得人取得该动产所有权。\n\n**第四百七十五条** 占有物毁损、灭失的，占有人应当承担赔偿责任。\n占有人善意占有物的，不承担赔偿责任。\n占有人恶意占有物的，应当承担赔偿责任。\n\n**第四百七十六条** 占有被侵夺的，占有人有权请求返还原物。\n占有物毁损、灭失的，占有人有权请求赔偿损失。\n\n**第四百七十七条** 占有被侵夺的，占有人有权请求返还原物。\n占有物毁损、灭失的，占有人有权请求赔偿损失。\n\n**第四百七十八条** 占有物

In [26]:
question

'农村集体经济的财产怎么确权'

In [27]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
<context>
{context}
</context>
<question>
{question}
</question>
"""

In [28]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。\n<context>\n第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###\n第五章 占有\n\n**第四百七十一条** 占有是指对物事实上的控制和支配。\n\n**第四百七十二条** 占有可以分为直接占有和间接占有。\n直接占有是指直接对物进行控制和支配。\n间接占有是指通过他人对物进行控制和支配。\n\n**第四百七十三条** 占有的取得和消灭，适用本法有关物权设立和消灭的规定。\n\n**第四百七十四条** 占有人合法占有动产的，善意取得人取得该动产所有权。\n占有人非法占有动产的，善意取得人取得该动产所有权。\n\n**第四百七十五条** 占有物毁损、灭失的，占有人应当承担赔偿责任。\n占有人善意占有物的，不承担赔偿责任。\n占有人恶意占有物的，应当承担赔偿责任。\n\n**第四百七十六条** 占有被侵夺的，占有人有权请求返还原物。\n占有物毁损、灭失的，占有人有权请求赔偿损失。\n\n**第四百七十

In [29]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，虽然未直接提及“农村集体经济的财产确权”的具体条款，但可以从相关物权和合同规定中推导出以下法律依据和原则：

1. **物权登记原则**（第三百九十条）：  
   地役权等物权的设立需依法办理登记，未经登记不得对抗善意第三人。这一原则可类推适用于农村集体财产的确权，即需通过法定登记程序（如不动产登记）明确权属。

2. **合同约定与转让规则**（第五百四十八条至第五百五十六条）：  
   - 财产权利的转让或变更需经当事人协商一致（第五百四十八条），且内容约定需明确（第五百四十九条）。  
   - 集体财产的转让若涉及债权债务，需遵循债权转让（需通知债务人）或债务转让（需债权人同意）的规定（第五百五十条至第五百五十三条）。

3. **占有与善意取得**（第四百七十四条）：  
   - 合法占有的动产可通过善意取得制度确权。若农村集体财产被非法占有，善意第三人可能取得所有权，但恶意占有人需承担赔偿责任（第四百七十五条、第四百七十八条）。

4. **地役权的关联性**（第三百九十五条）：  
   农村土地权利（如土地承包经营权、宅基地使用权）转让时，附属的地役权一并转让。确权需注意这些关联权利的完整性。

**建议操作步骤**：  
- **登记确权**：通过不动产登记机构办理集体土地、房屋等财产的权属登记。  
- **合同明确**：涉及集体财产流转时，签订书面合同并明确权利义务（参考第三百八十九条）。  
- **争议解决**：若权属争议，占有人可请求返还原物或赔偿（第四百七十六条至第四百八十条）。  

注：具体确权流程需结合《民法典》其他章节及《土地管理法》等特别法规。
